https://www.kaggle.com/datasets/jessicali9530/lfw-dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install opencv-python
!pip install insightface
!pip install onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 18.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 39.5 MB/s eta 0:00:00
  Created wheel for insightface: filename=insightface-0.7.3-cp311-cp311-linux_x86_64.whl size=1064921 sha256=e78378b5efba60260b626a59ad9fa8b555626497e6ab70324ba2bca129621548
  Stored in directory: /root/.cache/pip/wheels/27/d8/22/f52d858d16cd06e7b2e6aad34a1777dcfaf000be833bbf8146
Successfully built insightface


In [5]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from itertools import combinations
from insightface.app import FaceAnalysis

In [6]:
# Set paths and threshold
IMAGE_ROOT = "/content/drive/MyDrive/lfw-deepfunneled"
CSV_PATH = "/content/drive/MyDrive/peopleDevTest.csv"
THRESHOLD = 0.6

# Load ArcFace model
face_app = FaceAnalysis(name='buffalo_l')
face_app.prepare(ctx_id=0)

# Load CSV with names and image counts
df = pd.read_csv(CSV_PATH)

# Get face embedding using ArcFace
def get_embedding(image_path):
   full_path = os.path.join(IMAGE_ROOT, image_path)
   img = cv2.imread(full_path)
   if img is None:
       return None
   faces = face_app.get(img)
   return faces[0].embedding if faces else None

# Evaluate same-person pairs

def evaluate(df):
    total = 0
    correct = 0
    false_negatives = 0
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Evaluating"):
        person = row['name']
        path = os.path.join(IMAGE_ROOT, person)
        if not os.path.exists(path):
            continue

        # Use up to 5 images per person
        images = sorted(os.listdir(path))[:5]
        if len(images) < 2:
            continue
        # Generate all pairs from the selected images
        pairs = list(combinations(images, 2))
        for img1, img2 in pairs:
            emb1 = get_embedding(os.path.join(person, img1))
            emb2 = get_embedding(os.path.join(person, img2))
            if emb1 is None or emb2 is None:
                continue

            # Cosine similarity between embeddings
            similarity = np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
            is_match = similarity > THRESHOLD
            total += 1
            if is_match:
                correct += 1
            else:
                false_negatives += 1
            print(f"{person}: {img1} vs {img2} → Similarity: {similarity:.3f} → {'Match' if is_match else 'No Match'}")

    # Summary results
    print("\nSummary:")
    print(f"Total Pairs: {total}")
    print(f"Correct Matches: {correct}")
    print(f"False Negatives: {false_negatives}")
    print(f"Accuracy: {correct / total:.4f}" if total else "No valid pairs evaluated.")

evaluate(df)

download_path: /root/.insightface/models/buffalo_l


100%|██████████| 281857/281857 [00:06<00:00, 45217.18KB/s]
/usr/local/lib/python3.11/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:118: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


Evaluating:   0%|          | 5/1711 [00:13<50:57,  1.79s/it]  

Abdullah_Gul: Abdullah_Gul_0001.jpg vs Abdullah_Gul_0002.jpg → Similarity: 0.619 → Match
Abdullah_Gul: Abdullah_Gul_0001.jpg vs Abdullah_Gul_0003.jpg → Similarity: 0.614 → Match
Abdullah_Gul: Abdullah_Gul_0001.jpg vs Abdullah_Gul_0004.jpg → Similarity: 0.620 → Match
Abdullah_Gul: Abdullah_Gul_0001.jpg vs Abdullah_Gul_0005.jpg → Similarity: 0.640 → Match
Abdullah_Gul: Abdullah_Gul_0002.jpg vs Abdullah_Gul_0003.jpg → Similarity: 0.781 → Match
Abdullah_Gul: Abdullah_Gul_0002.jpg vs Abdullah_Gul_0004.jpg → Similarity: 0.900 → Match
Abdullah_Gul: Abdullah_Gul_0002.jpg vs Abdullah_Gul_0005.jpg → Similarity: 0.716 → Match


...

Zhang_Ziyi: Zhang_Ziyi_0002.jpg vs Zhang_Ziyi_0003.jpg → Similarity: 0.693 → Match
Zhang_Ziyi: Zhang_Ziyi_0002.jpg vs Zhang_Ziyi_0004.jpg → Similarity: 0.686 → Match


Evaluating: 100%|█████████▉| 1707/1711 [1:37:10<00:18,  4.62s/it]

Zhang_Ziyi: Zhang_Ziyi_0003.jpg vs Zhang_Ziyi_0004.jpg → Similarity: 0.675 → Match
Zoran_Djindjic: Zoran_Djindjic_0001.jpg vs Zoran_Djindjic_0002.jpg → Similarity: 0.797 → Match
Zoran_Djindjic: Zoran_Djindjic_0001.jpg vs Zoran_Djindjic_0003.jpg → Similarity: 0.731 → Match
Zoran_Djindjic: Zoran_Djindjic_0001.jpg vs Zoran_Djindjic_0004.jpg → Similarity: 0.695 → Match
Zoran_Djindjic: Zoran_Djindjic_0002.jpg vs Zoran_Djindjic_0003.jpg → Similarity: 0.735 → Match
Zoran_Djindjic: Zoran_Djindjic_0002.jpg vs Zoran_Djindjic_0004.jpg → Similarity: 0.705 → Match


Evaluating: 100%|██████████| 1711/1711 [1:37:27<00:00,  3.42s/it]

Zoran_Djindjic: Zoran_Djindjic_0003.jpg vs Zoran_Djindjic_0004.jpg → Similarity: 0.663 → Match

Summary:
Total Pairs: 2071
Correct Matches: 1570
False Negatives: 501
Accuracy: 0.7581
